In [ ]:
import pandas as pd
from scipy.stats import ttest_ind
from long_covid.colors import flatuicolors
from long_covid import styling
from matplotlib import pyplot as plt
import datetime
import numpy as np
from scipy import stats


def trim_axs(axs, N):
    """
    Reduce *axs* to *N* Axes. All further Axes are removed from the figure.
    """
    axs = axs.flat
    for ax in axs[N:]:
        ax.remove()
    return axs[:N]

In [ ]:
# Global variables for styling
COLORS = [flatuicolors.pomegranate, flatuicolors.amethyst, flatuicolors.belizehole]
ALPHA_VALUES = [1., 1., 0.4]
MARKERS = ['o', 'd', 'p']
VITALS_IDS = [65, 9, 43]
SIGNIFICANCE_MARKER_POSITIONS = [(2.5, 0.3), (880, 440), (70, 8)]
ITEM_ORDER_IN_LEGEND = [2,3,0,1]

X_LABEL = 'Weeks since PCR-test'
Y_LABELS = ['Change in RHR [bpm]', 'Change in daily steps', 'Change in daily sleep\nduration [minutes]']
BAR_LABEL = 'Week of PCR-test'
    
# Globally accessible input data
COHORTS = pd.read_feather('../data/03_derived/user_cohorts.feather')
WEEKLY_DEVIATIONS = pd.read_feather('../data/03_derived/weekly_vital_deviations_per_user.feather')

DATA = pd.merge(WEEKLY_DEVIATIONS, COHORTS, left_on='userid', right_on='user_id').drop(columns='user_id') 

In [ ]:
from matplotlib.colors import to_rgba

f, axarr = plt.subplots(3, 2, sharex='col', sharey='row', figsize=(6, 6))

colors = [flatuicolors.pomegranate, flatuicolors.amethyst]

for i, cohort in enumerate(['unvaccinated', 'vaccinated']):
    users = COHORTS[COHORTS[cohort]].user_id.values
    c = colors[i]
    
    for j, vital in enumerate([65, 9, 43]):
        ax = axarr[j, i]
        #print(cohort)
        data = WEEKLY_DEVIATIONS[WEEKLY_DEVIATIONS.userid.isin(users) & (WEEKLY_DEVIATIONS.vitalid == vital)]
        
        if cohort == 'unvaccinated':
            data = data[(data.weeks_since_test >= -3) & (data.weeks_since_test <= 15)]
             
        if cohort == 'vaccinated':
            data = data[(data.weeks_since_test >= -3) & (data.weeks_since_test <= 12)]
        
        average = data.groupby('weeks_since_test').vital_change.mean()
        
        if cohort == 'unvaccinated':
            weeks = range(-3, 15)
        else:
            weeks = range(-3, 13)
        
        for week in weeks:
            bplot = ax.boxplot(
                data[data.weeks_since_test == week].vital_change.values, 
                positions=[week],
                patch_artist=True,
                widths=0.75,
                flierprops={'markeredgecolor': c, 'markersize': 2, 'alpha': 0.5},
                medianprops={'c': c, 'lw': 1.5},
                #whis = [5, 95]
            )
            #print(bplot['boxes'][0])
            bplot['boxes'][0].set_facecolor(to_rgba(c, 0.2))
            bplot['boxes'][0].set_edgecolor('k')
            #bplot['boxes'][0].set_alpha(0.2)
            #boxplot(data, notch=None, vert=None, patch_artist=None, widths=None)
        #ax.scatter(data.weeks_since_test, data.vital_change, alpha=0.1, s=4, c=c)
        #ax.plot(average.index, average.values, c=c, ls='-', lw=2, alpha=0.8)
        #print(ax.get_xticks())
        styling.hide_and_move_axis(ax)
                
axarr[0, 0].set_xticks(range(-2, 15, 2))
axarr[0, 1].set_xticks(range(-2, 13, 2))

axarr[0, 0].set_xticklabels(range(-2, 15, 2))
axarr[0, 1].set_xticklabels(range(-2, 13, 2))

for ax in axarr[-1, :]:
    ax.set_xlabel(X_LABEL)
    
for ax, label in zip(axarr[:, 0], Y_LABELS):
    ax.set_ylabel(label)

axarr[0, 0].set_title('Unvaccinated')
axarr[0, 1].set_title('Vaccinated')

plt.tight_layout()
plt.savefig('../output/si_figure8_dot_plot_vital_changes.pdf')

In [ ]:
def QQplot(cohort, color, vitalid, max_week, output_path):
    
    size = 10.5
    
    if max_week == 15:
        f, axarr = plt.subplots(5, 4, figsize=(size, size))
        axarr = trim_axs(axarr, 3 + max_week + 1)
    elif max_week == 12:
        f, axarr = plt.subplots(4, 4, figsize=(size, size * 0.83), sharex=True, sharey=True)

    users = COHORTS[COHORTS.unvaccinated].user_id.values
    data = WEEKLY_DEVIATIONS[WEEKLY_DEVIATIONS.userid.isin(users) & (WEEKLY_DEVIATIONS.vitalid == vitalid)]

    for week, ax in zip(range(-3, max_week+1), axarr.flatten()):
        stats.probplot(data[data.weeks_since_test == week].vital_change.values, dist="norm", plot=ax, rvalue=True)
        styling.hide_and_move_axis(ax)

        ax.set_ylabel("Sample quantiles")
        ax.set_xlabel("Normal theoretical quantiles")
        
        if week > 1:
            ax.set_title(f"{week} weeks since test", size=10)
        elif week < -1:
            week = abs(week)
            ax.set_title(f"{week} weeks before test", size=10)
        elif week == 1:
            ax.set_title(f"{week} week since test", size=10)
        elif week == -1:
            week = abs(week)
            ax.set_title(f"{week} week before test", size=10)            
        else:
            ax.set_title(f"Week of test", size=10)

        #pylab.show()

        ax.get_lines()[0].set_markerfacecolor(color)
        ax.get_lines()[0].set_markeredgecolor(color)
        ax.get_lines()[0].set_markersize(4)
        ax.get_lines()[1].set_color('k')
       
    plt.tight_layout()
    plt.savefig(output_path)

In [ ]:
QQplot('negative', flatuicolors.belizehole, 65, 15, '../output/qq_negative_rhr.pdf')
QQplot('negative', flatuicolors.belizehole, 9, 15, '../output/qq_negative_steps.pdf')
QQplot('negative', flatuicolors.belizehole, 43, 15, '../output/qq_negative_sleep.pdf')

QQplot('vaccinated', flatuicolors.amethyst, 65, 12, '../output/qq_vaccinated_rhr.pdf')
QQplot('vaccinated', flatuicolors.amethyst, 9, 12, '../output/qq_vaccinated_steps.pdf')
QQplot('vaccinated', flatuicolors.amethyst, 43, 12, '../output/qq_vaccinated_sleep.pdf')

QQplot('unvaccinated', flatuicolors.pomegranate, 65, 15, '../output/qq_unvaccinated_rhr.pdf')
QQplot('unvaccinated', flatuicolors.pomegranate, 9, 15, '../output/qq_unvaccinated_steps.pdf')
QQplot('unvaccinated', flatuicolors.pomegranate, 43, 15, '../output/qq_unvaccinated_sleep.pdf')